In [ ]:
from brain_lib import *
from helpful_functions import *

In [ ]:
from typing import Optional

import requests
from urllib.parse import urljoin
import time
import json
import os
import getpass
from pathlib import Path

import pandas as pd

from multiprocessing.pool import ThreadPool
from functools import partial

import tqdm

In [ ]:
# How to proceed?

# start session and login
# make a alpha list with parameter optimization
# run all alphas using multisimulator
# prettify results and display
# compare results
# get the final optimized alpha!!!

In [ ]:
s = start_session()

Complete biometrics authentication and press any key to continue: 
https://api.worldquantbrain.com/authentication/persona?inquiry=inq_HiqAGWnmwV5WocZFH6rzFg8M



In [ ]:
'''
def generate_alpha(
    regular: str,
    region: str = "USA",
    universe: str = "TOP3000",
    neutralization: str = "INDUSTRY",
    delay: int = 1,
    decay: int = 0,
    truncation: float = 0.08,
    nan_handling: str = "OFF",
    unit_handling: str = "VERIFY",
    pasteurization: str = "ON",
    visualization: bool = False,
):
'''

'\ndef generate_alpha(\n    regular: str,\n    region: str = "USA",\n    universe: str = "TOP3000",\n    neutralization: str = "INDUSTRY",\n    delay: int = 1,\n    decay: int = 0,\n    truncation: float = 0.08,\n    nan_handling: str = "OFF",\n    unit_handling: str = "VERIFY",\n    pasteurization: str = "ON",\n    visualization: bool = False,\n):\n'

In [ ]:
datasets_df = get_datasets(s)

KeyError: 'results'

In [ ]:
selected_datasets_df = datasets_df.query("""
        delay == 1 &\
        0.8 < coverage <= 1 &\
        0 < fieldCount < 1000000 &\
        region == 'USA' &\
        universe == 'TOP3000' &\
        0 <= userCount < 10000000 &\
        5 < valueScore < 10 &\
        name.str.contains('', case=False) == 1 & (category == 'news')
    """, engine='python')
# selected_datasets_df

In [ ]:
# MAKE JUST ALPHA :)

just_alphas = []


# for df_id in selected_datasets_df['id']:
#     datafields_df = get_datafields(s, dataset_id=df_id)
#     for para in range(len(datafields_df)):
#         name = datafields_df['id'][para]
#         typ = datafields_df['type'][para]
#         if typ == 'VECTOR':
#             name = 'vec_avg(' + name + ')'
#         aa = f'''
# signal = group_backfill(group_rank(ts_rank(mdl169_backfill_cpm-close, {d1}), subindustry),subindustry, {d2});
# returns > -{drop} ? signal : -1
#         '''
#         just_alphas.append(aa)


for d1 in [60]:
    for d2 in [20]:
        for cpl in ['cpl', 'cpm']:
            for grp in ['subindustry', 'industry', 'country']:
                for rett in ['-high/low']:
                    aa = f'''
        signal = group_backfill(group_rank(ts_rank(mdl169_backfill_{cpl}-{'close'}, {d1}), {grp}),{grp}, {d2});
        returns > -{0.11} ? signal : {rett}
                '''
                    just_alphas.append(aa)



In [ ]:
# MAKE ALPHA LIST !!!

# possible_settings = {
#     'universe': ['TOP3000', 'TOP1000', 'TOP500', 'TOP200', 'MINVOL1M'],
#     'neutralization': ['NONE', 'MARKET', 'SECTOR', 'INDUSTRY', 'SUBINDUSTRY', 'SLOW', 'FAST', 'SLOW_AND_FAST'],
#     'decay': [0,1,5,10,20,60,120,252],
#     'truncation': [0.01, 0.03, 0.05, 0.08]
# }


k = []
for a1 in just_alphas:
    for decay in [3,5,7,10]:
        default_settings = {
            'universe': 'TOP3000',
            'neutralization': 'SLOW',
            'decay': decay,
            'truncation': 0.04,
        }
        k.append((a1, default_settings))


# default_settings = {
#     'universe': 'TOP3000',
#     'neutralization': 'SUBINDUSTRY',
#     'decay': 3,
#     'truncation': 0.03
# }

# k = [(x, default_settings) for x in just_alphas]


In [ ]:
alpha_list = [generate_alpha(x,
                             universe=settings['universe'],
                             neutralization=settings['neutralization'],
                             decay=settings['decay'],
                             truncation=settings['truncation'],
                             ) for (x,settings) in k]

# NEXT TASK IS TO CHANGE SETTINGS OF SINGLE ALPHA AND CHECK !!!

In [ ]:
len(alpha_list)

216

In [ ]:
results = simulate_alpha_list_multi(s, alpha_list)
prettify_result(results, detailed_tests_view=True)

100%|██████████| 72/72 [1:51:49<00:00, 93.18s/it]   


,pnl,book_size,long_count,short_count,turnover,returns,drawdown,margin,fitness,sharpe,...,low_fitness,low_sharpe,low_sub_universe_sharpe,low_turnover,matches_competition,matches_themes,prod_correlation,regular_submission,self_correlation,units
0,8017622,20000000,1545,1556,0.2068,0.0815,0.0348,0.000788,2.29,3.64,...,"{'limit': 1.0, 'result': 'PASS', 'value': 2.29}","{'limit': 1.58, 'result': 'PASS', 'value': 3.64}","{'limit': 1.58, 'result': 'PASS', 'value': 1.72}","{'limit': 0.01, 'result': 'PASS', 'value': 0.2...","{'limit': nan, 'result': 'PASS', 'value': nan}","{'limit': nan, 'result': 'WARNING', 'value': nan}","{'limit': nan, 'result': 'PENDING', 'value': nan}","{'limit': nan, 'result': 'PENDING', 'value': nan}","{'limit': nan, 'result': 'PENDING', 'value': nan}","{'limit': nan, 'result': 'WARNING', 'value': nan}"
1,8017635,20000000,1545,1556,0.2068,0.0815,0.0348,0.000788,2.29,3.64,...,"{'limit': 1.0, 'result': 'PASS', 'value': 2.29}","{'limit': 1.58, 'result': 'PASS', 'value': 3.64}","{'limit': 1.58, 'result': 'PASS', 'value': 1.72}","{'limit': 0.01, 'result': 'PASS', 'value': 0.2...","{'limit': nan, 'result': 'PASS', 'value': nan}","{'limit': nan, 'result': 'WARNING', 'value': nan}","{'limit': nan, 'result': 'PENDING', 'value': nan}","{'limit': nan, 'result': 'PENDING', 'value': nan}","{'limit': nan, 'result': 'PENDING', 'value': nan}","{'limit': nan, 'result': 'WARNING', 'value': nan}"
2,8017605,20000000,1545,1556,0.2068,0.0815,0.0348,0.000788,2.29,3.64,...,"{'limit': 1.0, 'result': 'PASS', 'value': 2.29}","{'limit': 1.58, 'result': 'PASS', 'value': 3.64}","{'limit': 1.58, 'result': 'PASS', 'value': 1.72}","{'limit': 0.01, 'result': 'PASS', 'value': 0.2...","{'limit': nan, 'result': 'PASS', 'value': nan}","{'limit': nan, 'result': 'WARNING', 'value': nan}","{'limit': nan, 'result': 'PENDING', 'value': nan}","{'limit': nan, 'result': 'PENDING', 'value': nan}","{'limit': nan, 'result': 'PENDING', 'value': nan}","{'limit': nan, 'result': 'WARNING', 'value': nan}"
3,7045652,20000000,1540,1561,0.1522,0.0716,0.0345,0.000941,2.28,3.33,...,"{'limit': 1.0, 'result': 'PASS', 'value': 2.28}","{'limit': 1.58, 'result': 'PASS', 'value': 3.33}","{'limit': 1.44, 'result': 'PASS', 'value': 1.53}","{'limit': 0.01, 'result': 'PASS', 'value': 0.1...","{'limit': nan, 'result': 'PASS', 'value': nan}","{'limit': nan, 'result': 'WARNING', 'value': nan}","{'limit': nan, 'result': 'PENDING', 'value': nan}","{'limit': nan, 'result': 'PENDING', 'value': nan}","{'limit': nan, 'result': 'PENDING', 'value': nan}","{'limit': nan, 'result': 'WARNING', 'value': nan}"
4,7152789,20000000,1539,1562,0.1511,0.0727,0.0326,0.000962,2.28,3.29,...,"{'limit': 1.0, 'result': 'PASS', 'value': 2.28}","{'limit': 1.58, 'result': 'PASS', 'value': 3.29}","{'limit': 1.42, 'result': 'PASS', 'value': 1.55}","{'limit': 0.01, 'result': 'PASS', 'value': 0.1...","{'limit': nan, 'result': 'PASS', 'value': nan}","{'limit': nan, 'result': 'WARNING', 'value': nan}","{'limit': nan, 'result': 'PENDING', 'value': nan}","{'limit': nan, 'result': 'PENDING', 'value': nan}","{'limit': nan, 'result': 'PENDING', 'value': nan}","{'limit': nan, 'result': 'WARNING', 'value': nan}"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
211,6230957,20000000,1544,1557,0.1638,0.0633,0.0278,0.000773,1.95,3.14,...,"{'limit': 1.0, 'result': 'PASS', 'value': 1.95}","{'limit': 1.58, 'result': 'PASS', 'value': 3.14}","{'limit': 1.36, 'result': 'PASS', 'value': 1.38}","{'limit': 0.01, 'result': 'PASS', 'value': 0.1...","{'limit': nan, 'result': 'PASS', 'value': nan}","{'limit': nan, 'result': 'WARNING', 'value': nan}","{'limit': nan, 'result': 'PENDING', 'value': nan}","{'limit': nan, 'result': 'PENDING', 'value': nan}","{'limit': nan, 'result': 'PENDING', 'value': nan}","{'limit': nan, 'result': 'WARNING', 'value': nan}"
212,6229809,20000000,1544,1557,0.1638,0.0633,0.0278,0.000773,1.95,3.14,...,"{'limit': 1.0, 'result': 'PASS', 'value': 1.95}","{'limit': 1.58, 'result'

In [ ]:
table = prettify_result(results, detailed_tests_view=True)


In [ ]:
def performance_comparison(s, alpha_id, team_id:Optional[str] = None, competition:Optional[str] = 'IIITTM12.0'):
    """
    Returns performance comparison for merged performance check
    """
    if competition is not None:
        part_url = f'competitions/{competition}'
    elif team_id is not None:
        part_url = f'teams/{team_id}'
    else:
        part_url = 'users/self'
    while True:
        result = s.get(
            f"https://api.worldquantbrain.com/{part_url}/alphas/" + alpha_id + "/before-and-after-performance"
        )
        if "retry-after" in result.headers:
            time.sleep(float(result.headers["Retry-After"]))
        else:
            break
    if result.json().get("stats", 0) == 0:
        return {}
    if result.status_code != 200:
        return {}

    return result.json()

In [ ]:
def get_simulation_result_json(s, alpha_id):
    return s.get("https://api.worldquantbrain.com/alphas/" + alpha_id).json()

In [ ]:
def check_passes(i):
    for test in [0,1,2,3,4,5,11,12]:
        if i['is_tests']['result'][test] == 'FAIL':
            return 0
    return 1




In [ ]:
p=[]
res=[]
for i in results:
    a = performance_comparison(s, i['alpha_id'])
    # corr = check_prod_corr_test(s, i['alpha_id'])['value'][0]
    p.append(a)
    res.append((i['alpha_id'], a['score']['after'] - a['score']['before'], check_passes(i)))

In [ ]:
res

[('YNM3zEJ', 4482.0, 0),
 ('2vmV668', 4782.0, 0),
 ('EqRnzz0', 3020.0, 1),
 ('ZjAVzVx', 2628.0, 1),
 ('JnQazax', 3453.0, 1),
 ('XkJZzZa', 3756.0, 0),
 ('Mb3vzVM', 4016.0, 0),
 ('pn5bG3g', 3164.0, 1),
 ('On8dzgJ', 4069.0, 1),
 ('gmZNL0g', 4139.0, 1),
 ('bNO0Agp', 4597.0, 0),
 ('EqRnxMK', 4957.0, 0),
 ('EqRn2or', 2946.0, 1),
 ('6RK62gG', 3717.0, 1),
 ('pn5bpJq', 4027.0, 1),
 ('5LgK2lM', 4255.0, 0),
 ('NnVE277', 3232.0, 1),
 ('pn5bpjv', 4154.0, 1),
 ('NnVElQo', 4322.0, 1),
 ('aNxm9LO', 4752.0, 0),
 ('onWj066', 5050.0, 0),
 ('aNx03kv', 2628.0, 1),
 ('LnZvxLn', 3453.0, 1),
 ('onWbem6', 3756.0, 0),
 ('YNM3o6l', 4536.0, 0),
 ('7x6PRA8', 4783.0, 0),
 ('QnKRpJ5', 3257.0, 1),
 ('VX0rK95', 4016.0, 0),
 ('0ArVnl1', 3164.0, 1),
 ('kngwerl', 4069.0, 1),
 ('l7KnJj2', 4482.0, 0),
 ('qn0aejj', 4782.0, 0),
 ('LnZvO7m', 3020.0, 1),
 ('Knroxaj', 4139.0, 1),
 ('Mb3E9m8', 4597.0, 0),
 ('bNOm7ER', 4957.0, 0),
 ('AnvpYLW', 2946.0, 1),
 ('xebL9lw', 3738.0, 1),
 ('ZjA8qXj', 4027.0, 1),
 ('GnO2YYQ', 4255.0, 0),


In [ ]:
resss = []
for i in ids:
    a = performance_comparison(s, i)
    corr = check_prod_corr_test(s, i)['value'][0]
    resss.append((i, a['score']['after'] - a['score']['before'], check_passes(ids.index(i)), corr))

AttributeError: 'DataFrame' object has no attribute 'alphas'

In [ ]:
get_prod_corr(s, 'On8gzkR')

""


In [ ]:
for i in res:
    if i[2]==1:
        print(i)

('EqRnzz0', 3020.0, 1)
('ZjAVzVx', 2628.0, 1)
('JnQazax', 3453.0, 1)
('pn5bG3g', 3164.0, 1)
('On8dzgJ', 4069.0, 1)
('gmZNL0g', 4139.0, 1)
('EqRn2or', 2946.0, 1)
('6RK62gG', 3717.0, 1)
('pn5bpJq', 4027.0, 1)
('NnVE277', 3232.0, 1)
('pn5bpjv', 4154.0, 1)
('NnVElQo', 4322.0, 1)
('aNx03kv', 2628.0, 1)
('LnZvxLn', 3453.0, 1)
('QnKRpJ5', 3257.0, 1)
('0ArVnl1', 3164.0, 1)
('kngwerl', 4069.0, 1)
('LnZvO7m', 3020.0, 1)
('Knroxaj', 4139.0, 1)
('AnvpYLW', 2946.0, 1)
('xebL9lw', 3738.0, 1)
('ZjA8qXj', 4027.0, 1)
('onWbAAb', 3232.0, 1)
('ZjA8qxZ', 4154.0, 1)
('pn52o9g', 3257.0, 1)
('PngRl2M', 4322.0, 1)
('nnPjOJ3', 2628.0, 1)
('AnvpOWQ', 3453.0, 1)
('PngRXG7', 3164.0, 1)
('pn521N3', 4069.0, 1)
('3EVKEbX', 3020.0, 1)
('nnPjnvx', 4139.0, 1)
('RN6OdQo', 2946.0, 1)
('ZjA8reY', 3717.0, 1)
('JnQAbZm', 4027.0, 1)
('GnO2wP5', 3232.0, 1)
('dn1MxXw', 4154.0, 1)
('AnvpwPR', 3257.0, 1)
('YNMVpMA', 4322.0, 1)
('VX0r3a5', 2746.0, 1)
('WNeZVbZ', 3567.0, 1)
('xebLNKN', 3258.0, 1)
('1oZ8pqX', 4184.0, 1)
('onWbNOl',

In [ ]:
for i in results:
    print(i)
    break

{'alpha_id': 'QnK6zVp', 'simulate_data': {'type': 'REGULAR', 'settings': {'instrumentType': 'EQUITY', 'region': 'USA', 'universe': 'TOP3000', 'delay': 1, 'decay': 5, 'neutralization': 'SLOW', 'truncation': 0.08, 'pasteurization': 'ON', 'unitHandling': 'VERIFY', 'nanHandling': 'OFF', 'language': 'FASTEXPR', 'visualization': False}, 'regular': '\n            signal = group_backfill(group_rank(ts_rank(mdl169_backfill_cpl-close, 60), industry),industry, 20);\n            returns > -0.09 ? signal : returns\n                    '}, 'is_stats':        pnl  bookSize  longCount  shortCount  turnover  returns  drawdown  \
0  7708484  20000000       1539        1562    0.2014   0.0784    0.0363   

     margin  fitness  sharpe alpha_id  
0  0.000778     2.25    3.61  QnK6zVp  , 'pnl': None, 'stats': None, 'is_tests':                        name   result  limit   value  \
0                LOW_SHARPE     PASS   1.58  3.6100   
1               LOW_FITNESS     PASS   1.00  2.2500   
2              LO

In [ ]:
import pandas as pd

df = pd.DataFrame(res)
df

In [ ]:
results[0]

{'alpha_id': 'nnPQgWl',
 'simulate_data': {'type': 'REGULAR',
  'settings': {'instrumentType': 'EQUITY',
   'region': 'USA',
   'universe': 'TOP3000',
   'delay': 1,
   'decay': 1,
   'neutralization': 'SLOW',
   'truncation': 0.08,
   'pasteurization': 'ON',
   'unitHandling': 'VERIFY',
   'nanHandling': 'OFF',
   'language': 'FASTEXPR',
   'visualization': False},
  'regular': '\nsignal = group_backfill(group_rank(ts_rank(mdl169_backfill_cpl-close, 60), industry),industry, 20);\nreturns > -0.11 ? signal : -1\n        '},
 'is_stats':        pnl  bookSize  longCount  shortCount  turnover  returns  drawdown  \
 0  9341948  20000000       1555        1546     0.431    0.095    0.0315   
 
      margin  fitness  sharpe alpha_id  
 0  0.000441     1.86    3.97  nnPQgWl  ,
 'pnl': None,
 'stats': None,
 'is_tests':                        name   result  limit  value  \
 0                LOW_SHARPE     PASS   1.58  3.970   
 1               LOW_FITNESS     PASS   1.00  1.860   
 2           

In [ ]:
check_prod_corr_test(s, 'XkJe9M5')['value'][0]

0.8

In [ ]:
results[1]

{'alpha_id': 'VX0ezPA',
 'simulate_data': {'type': 'REGULAR',
  'settings': {'instrumentType': 'EQUITY',
   'region': 'USA',
   'universe': 'TOP3000',
   'delay': 1,
   'decay': 6,
   'neutralization': 'SLOW',
   'truncation': 0.08,
   'pasteurization': 'ON',
   'unitHandling': 'VERIFY',
   'nanHandling': 'OFF',
   'language': 'FASTEXPR',
   'visualization': False},
  'regular': '\n            signal = group_backfill(group_rank(ts_rank(mdl169_backfill_cpl-close, 60), industry),industry, 20);\n            returns > -0.09 ? signal : returns\n                    '},
 'is_stats':        pnl  bookSize  longCount  shortCount  turnover  returns  drawdown  \
 0  7500995  20000000       1539        1562    0.1864   0.0763    0.0363   
 
      margin  fitness  sharpe alpha_id  
 0  0.000818     2.26    3.53  VX0ezPA  ,
 'pnl': None,
 'stats': None,
 'is_tests':                        name   result  limit   value  \
 0                LOW_SHARPE     PASS   1.58  3.5300   
 1               LOW_FITN

In [ ]:
ids = []
for i in results:
    ids.append(i['alpha_id'])

In [ ]:
ids

['QnK6zVp',
 'VX0ezPA',
 '99WkMbr',
 '99WkMbr',
 'nnPQn63',
 'ZjANzq1',
 'On8gzkR',
 '6RKN015',
 'NnV8zjo',
 'nnPQn63',
 'ZjANzq1',
 'On8gzkR',
 '6RKN015',
 'NnV8zjo',
 'QnK6zVp',
 'VX0ezPA',
 '99WkMbr',
 'nnPQn63',
 '8OPXZ6q',
 '5LgY166',
 'Mb3Vn8k',
 'ZjANzq1',
 'On8gzkR',
 '6RKN015',
 'NnV8zjo',
 'QnK6zVp',
 'VX0ezPA',
 '7x6lP9Q',
 'dn1vJLx',
 '8OPXZ6q',
 '5LgY166',
 'Mb3Vn8k',
 'QnK6RqQ',
 'QnK6RqQ',
 'gmZLNwQ',
 'YNMY3nJ',
 'gmZLNwQ',
 'YNMY3nJ',
 '7x6lP9Q',
 'dn1vJLx',
 '8OPXZ6q',
 '5LgY166',
 'Mb3Vn8k',
 'QnK6RqQ',
 'gmZLNwQ',
 'YNMY3nJ',
 '7x6lP9Q',
 'dn1vJLx',
 'LnZ2Pj1',
 '8OP2moV',
 '0Arabl2',
 'EqR2Z7K',
 '3EV2Mb0',
 'WNexEqx',
 'PngA09q',
 'kngYqlP',
 'LnZ2Pj1',
 '8OP2moV',
 '0Arabl2',
 'EqR2Z7K',
 '3EV2Mb0',
 'WNexEqx',
 'PngA09q',
 'kngYqlP',
 'LnZ2Pj1',
 '8OP2moV',
 '0Arabl2',
 'EqR2Z7K',
 '3EV2Mb0',
 'WNexEqx',
 'PngA09q',
 'kngYqlP',
 'l7KNOOl',
 'nnPXRRd',
 'bNOKZZ6',
 'RN6vQEe',
 'onWk5Ek',
 'LnZ2aMv',
 'xebE8Vw',
 'gmZ0OXM',
 'l7KNOOl',
 'nnPXRRd',
 'bNOKZZ6',
 'RN